In [36]:
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn import tree
from sklearn import preprocessing

titanic_dataset = 'docs/titanic.csv'
breast_cancer_dataset = 'docs/breastCancerDataset.csv'

titanic = pd.read_csv(titanic_dataset, header=0, delimiter=',', names=['Pclass', 'sex', 'age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Initial', 
'Age_band', 'Family_size', 'Alone', 'Fare_cat', 'Deck', 'Title', 'Is_Married', 'Survived']) # se lee el csv y 
                                                                                            #se indican el nombre de las columnas

cancer = pd.read_csv(breast_cancer_dataset, header=0, delimiter=',', names=['mean radius', 'mean texture', 'mean perimeter', 'mean area', 
'mean smoothness', 'mean compactness', 'mean concavity', 'mean concave points', 'mean symmetry', 'mean fractal dimension', 'radius error', 
'texture error', 'perimeter error', 'area error', 'smoothness error', 'compactness error', 'concavity error', 'concave points error', 
'symmetry error', 'fractal dimension error', 'worst radius', 'worst texture', 'worst perimeter', 'worst area', 'worst smoothness', 
'worst compactness', 'worst concavity', 'worst concave points', 'worst symmetry', 'worst fractal dimension', 'diagnosis'])

atributos_titanic = titanic.loc[:, 'Pclass':'Is_Married'] # selección de las columnas de atributos del dataset titanic
objetivo_titanic = titanic['Survived'] # selección de la columna objetivo del dataset titanic

atributos_cancer = cancer.loc[:, 'mean radius':'worst fractal dimension']
objetivo_cancer = cancer['diagnosis']

In [82]:
def metodo_evaluacion_robusta(dataset, atributos, objetivo, N_EXP, CV):
    
    '''
    Devuelve la media de las tasas de aciertos del algoritmo obtenidas en cada iteracion
    '''

    codificador_atributos = preprocessing.OrdinalEncoder() # Codificador adecuado para los atributos
    codificador_atributos.fit(atributos)  # Ajusta la codificación a los atributos
    atributos_codificados = codificador_atributos.transform(atributos)

    codificador_objetivo = preprocessing.LabelEncoder() # Codificador adecuado para el objetivo
    objetivo_codificado = codificador_objetivo.fit_transform(objetivo) # El método fit_transform ajusta la codificación y la aplica a los datos
    
    atributos_entrenamiento, atributos_prueba, objetivo_entrenamiento, objetivo_prueba = model_selection.train_test_split(
    atributos_codificados, objetivo_codificado,  # Conjuntos de datos a dividir, usando los mismos índices para ambos
    random_state=12345,  # Valor de la semilla aleatoria, para que el muestreo sea reproducible, a pesar de ser aleatorio
    test_size=.20  # Tamaño del conjunto de prueba
    )  

    clasif_arbol_decision = tree.DecisionTreeClassifier() # creamos el clasificador
    clasif_arbol_decision.fit(X=atributos_entrenamiento, y=objetivo_entrenamiento) 
    
    lista_promedios = [] # Inicializamos una lista, en la que iremos metiendo los promedios

    for i in range(N_EXP):
        scores = model_selection.cross_val_score(estimator=clasif_arbol_decision, X=atributos_prueba, y=objetivo_prueba, cv=CV, scoring='balanced_accuracy')
        # La variable scores almacena la tasa de aciertos del algoritmo sobre los datos que le pasamos como parámetros
        promedio = scores.mean() # Calculamos el promedio como la media los scores
        lista_promedios.append(promedio) #Añadimos el promedio a la lista de promedios
    
    media = sum(lista_promedios)/len(lista_promedios) # Calculamos la media de los promedios almacenados en la lista, que es 
                                                        # el dato que devolvera este método
    return media
    

In [83]:
metodo_evaluacion_robusta(cancer, atributos_cancer, objetivo_cancer, 10, 10)

0.8887500000000002

In [84]:
def algoritmo_sfs(dataset, D):
    
    '''
    Devuelve las K combinaciones de variables que proporcionan mayor rendimiento, acompañadas de su rendimiento y tamaño.
    Todo esto mostrado en una tabla
    '''
    
    solucion_actual = []    # Inicializamos la lista en la que almacenaremos la solucion actual
    lista_auxiliar = []     # Inicializamos una lista auxiliar
    solucion = []           # Esta lista se mostrara en la tabla. Contendra las mejores combinaciones de variables
    lista_mejores_promedios = [] # Esta lista se mostrara en la tabla. Contendra los mejores promedios
    size = []               # Esta lista se mostrara en la tabla. Contendra los K tamaños de la lista solucion
    k=0                     # inicializamos K
    variables_predictoras = dataset.columns.tolist() # Metemos en variables_predictoras los nombres de todas las columnas del dataset
    nombre_objetivo = variables_predictoras.pop(len(variables_predictoras)-1) # Eliminamos de variables predictoras el nombre de la columna objetivo
                                                                    # Y almacenamos en nombre_objetivo dicho nombre
    objetivo = dataset[nombre_objetivo] # Obtenemos la columna objetivo del dataset
    variables_sin_añadir = variables_predictoras # en variables sin añadir almacenamos los nombres de todas las 
                                                # columnas del dataset, exceptuando la del objetivo, eliminada anteriormente
    i=1
    while k < D:
        lista_scores = [] #Inicializamos 3 listas mas, necesarias para que todos los datos se muestren en la tabla correctamente
        lista_sol = []
        solucion_auxiliar = []
        for v in variables_sin_añadir: # recorremos las variables que aun no han sido añadidas
            lista_sol = solucion_actual
            lista_sol.append(dataset[v]) # Añadimos a lista_sol la columna del dataset cuyo nombre es la variable v
            solucion_temporal = lista_sol # Pasamos a solucion_temporal la lista_sol
            solucion_temporal = np.reshape(np.ravel(lista_sol), (len(objetivo),i+k)) # Transformamos el tamaño de la solucion 
                                                                 # temporal para que la solucion temporal tenga el mismo numero
                                                                 # de filas que el objetivo
            score = metodo_evaluacion_robusta(dataset, solucion_temporal, objetivo, 1, 3) # Calculamos el score
            lista_scores.append(score) # Añadimos el score a la lista de scores
            i=i+1 # Aumentamos i
        mejor_promedio = np.amax(lista_scores) # Calculamos el mejor score de la lista de scores
        mejor_solucion_temporal = variables_sin_añadir[lista_scores.index(mejor_promedio)] # La mejor solucion temporal sera la
                                                                 # variable que haya obtenido el mejor promedio
            
        solucion_actual.append(dataset[mejor_solucion_temporal]) # Añadimos a solucion actual la columna del dataset cuyo nombre
                                                                 # es la mejor solucion temporal
        lista_auxiliar.append(mejor_solucion_temporal) # Añadimos dicha variable a una lista auxiliar
        solucion_auxiliar = solucion_auxiliar + lista_auxiliar # Añadimos la lista auxiliar a otra lista para no tener problemas
                                                               # a la hora de mostrar la tabla
        solucion.append(solucion_auxiliar) # Añadimos a la lista solucion la lista solucion_auxiliar
        lista_mejores_promedios.append(mejor_promedio) # Añadimos a la lista de mejores promedios el mejor promedio
        size.append(len(solucion))   # Añadimos a la lista size el tamaño de la lista solucion
        variables_sin_añadir.remove(variables_sin_añadir[lista_scores.index(mejor_promedio)]) # Borramos de variables_sin_añadir
                                                                                              # la variable añadida
        k=k+1 # Aumentamos K

    print("\nTabla respuesta:\n")
    for j in range (0,D):
        frame_data={'solution':solucion, 'score':lista_mejores_promedios, 'size':size}
    df=pd.DataFrame(frame_data)

    return df.sort_values(by=['score'], ascending=False)

In [85]:
algoritmo_sfs(titanic, 5)


Tabla respuesta:



,solution,score,size
0,[Pclass],0.613810,1
3,"[Pclass, Family_size, Parch, Initial]",0.586071,4
1,"[Pclass, Family_size]",0.573690,2
4,"[Pclass, Family_size, Parch, Initial, Fare_cat]",0.557619,5
2,"[Pclass, Family_size, Parch]",0.540754,3
